<a href="https://colab.research.google.com/github/Maraudier/tkfruit/blob/master/BF_inceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Colab library to upload files to notebook
from google.colab import files

# Install Kaggle library
!pip install kaggle

In [0]:
!mkdir .kaggle
!mkdir ~/.kaggle

In [0]:
import json
token = {"username":"williamahtou","key":"40faa6e3f4d012c9b39cf9e96a731583"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!chmod 600 /content/.kaggle/kaggle.json

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [0]:
!kaggle config set -n path -v{/content}

In [0]:
!kaggle datasets download -d sriramr/fruits-fresh-and-rotten-for-classification -p /content

In [0]:
!unzip \*.zip

In [0]:
!rm -rf /content/dataset/dataset/

In [0]:
!ls

In [0]:
import pathlib
# Loading data set
train_dir = pathlib.Path('dataset/train')
test_dir = pathlib.Path('dataset/test')

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Version of tensorflow should be specified in Colab
%tensorflow_version 2.x
import tensorflow as tf
AUTOTUNE = tf.data.experimental.AUTOTUNE
from tensorflow.keras import datasets, layers, models
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

In [0]:
image_count = len(list(train_dir.glob('*/*.png'))) + len(list(test_dir.glob('*/*.png')))
CLASS_NAMES = np.array([item.name for item in train_dir.glob('*') if item.name != 'LICENSE.txt'])
CLASS_NAMES

In [0]:
# The 1./255 is to convert from uint8 to float32 in range [0,1]. Split data into 80/20 
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
# The transformations shear_range, zoom_range, horizontal_flip will improve accuracy across data and allow for more generalized inputs
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
# CONSTANTS, 299x299 is the standard size for this particular algorithm (inceptionV3)
BATCH_SIZE = 128
IMG_HEIGHT = 299
IMG_WIDTH = 299
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)


In [0]:

train_data_gen = train_generator.flow_from_directory(directory=str(train_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode="sparse",
                                                     classes = list(CLASS_NAMES))
test_data_gen = test_generator.flow_from_directory(directory=str(test_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode="sparse",
                                                     classes = list(CLASS_NAMES))


In [0]:
model = tf.keras.applications.InceptionV3()
new_input = model.input
hidden_layer = model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

In [0]:
image_features_extract_model.summary()

In [0]:
# Compiler
image_features_extract_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
#tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

# Training the data and validating with test data
# CURRENTLY takes quite a while to train (around an hour)
history = image_features_extract_model.fit_generator(
        train_data_gen,
        epochs=10,
        validation_data=test_data_gen)


In [0]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.1, 1])
plt.legend(loc='lower right')

test_loss, test_acc = image_features_extract_model.evaluate(test_data_gen, verbose=2)



In [0]:
sum(history.history['val_accuracy'])/10
sum(history.history['accuracy'])/10

In [0]:

# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir logs

In [0]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances